In [58]:
!pip3 install -U scikit-learn scipy matplotlib

     ---------------------------------------- 7.5/7.5 MB 20.1 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 30.8 MB/s eta 0:00:00
     ---------------------------------------- 298.0/298.0 kB ? eta 0:00:00
     -------------------------------------- 163.6/163.6 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\윤성주\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\~atplotlib\\ft2font.cp310-win_amd64.pyd'
Check the permissions.



In [1]:
import pandas as pd

df = pd.read_csv(r'../train_data_new_filled.csv')

In [2]:
df.head()

,나이,BMI,수술시간(min),출혈량(mL),GAS OUT,SD start,병변1 크기(Cm),margin(d),WBCmean,CRPmean,...,invasion,radicality,LN dissection,수혈,위암병소수,병변1 tubular,병변1 circular,Stage,DSL,onset
0,53.0,17.11,180.0,80.0,3.0,4.0,10.4,1.3,7.998220,5.44,...,1.0,0.0,4.0,0.0,1.0,0.0,2.0,4.0,0.0,NaN
1,62.0,24.01,180.0,80.0,2.0,5.0,2.3,9.6,7.941018,5.06,...,1.0,0.0,4.0,0.0,1.0,3.0,2.0,1.0,0.0,NaN
2,70.0,29.70,180.0,50.0,2.0,5.0,1.5,14.8,8.377507,10.18,...,1.0,0.0,2.0,0.0,1.0,2.0,2.0,1.0,0.0,NaN
3,72.0,25.06,180.0,100.0,4.0,14.0,1.5,13.1,9.600000,16.45,...,1.0,0.0,4.0,0.0,2.0,1.0,0.0,1.0,0.0,NaN
4,53.0,27.88,180.0,50.0,3.0,4.0,1.2,11.6,9.360000,10.54,...,1.0,0.0,4.0,0.0,2.0,1.0,3.0,1.0,0.0,NaN


In [11]:
numeric_columns = (df.iloc[:,0:9])

# Standarize numeric values
for name in numeric_columns.columns:
    numeric_columns[name] = (numeric_columns[name] - numeric_columns[name].mean()) / numeric_columns[name].std()
    
numeric_columns_with_result = pd.concat([numeric_columns, df.iloc[:,[-2]]],axis=1)


categorical_columns = (df.iloc[:,10:-2])
categorical_columns_with_result = pd.concat([categorical_columns,df.iloc[:,[-2]]],axis=1)

C:\Users\윤성주\AppData\Local\Temp\ipykernel_34580\4244783022.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_columns[name] = (numeric_columns[name] - numeric_columns[name].mean()) / numeric_columns[name].std()


In [16]:
# calc correlation with numerical columns

correlation1 = numeric_columns_with_result.corrwith(numeric_columns_with_result["DSL"])
dict_corr1 = correlation1.dropna().abs().to_dict()
sorted_dict_corr1 = dict(sorted(dict_corr1.items(), key=lambda item: item[1]))

In [17]:
sorted_dict_corr1

{'수술시간(min)': 0.002717737704227472,
 'margin(d)': 0.008533165533053557,
 'SD start': 0.025754225044157416,
 'GAS OUT': 0.02923031264886944,
 '나이': 0.0329802017053908,
 'BMI': 0.048025137167642486,
 'WBCmean': 0.06580052842192045,
 '출혈량(mL)': 0.0697330777457113,
 '병변1 크기(Cm)': 0.07100133801833368,
 'DSL': 1.0}

In [19]:
# calc correlation with numerical columns

correlation2 = categorical_columns_with_result.corrwith(categorical_columns_with_result["DSL"])
dict_corr2 = correlation2.dropna().abs().to_dict()
sorted_dict_corr2 = dict(sorted(dict_corr2.items(), key=lambda item: item[1]))

In [20]:
sorted_dict_corr2

{'smoking': 0.014600014121050268,
 '수혈': 0.01696512417726904,
 '가족암병력': 0.018327365198701345,
 '성별': 0.020439125265294567,
 '병변1 circular': 0.020830321282751543,
 '병변1 tubular': 0.02429538296322506,
 'LN dissection ': 0.02639152904168625,
 '정규/응급': 0.03374583467306714,
 'adhesion': 0.037017786472607075,
 '위암병소수': 0.04439782515689469,
 'invasion': 0.04819768966264102,
 '수술명': 0.06072471119956728,
 '개복전환': 0.07061226398538885,
 '문합법': 0.0935310972020782,
 '합병절제                   ': 0.09627777194224613,
 'Stage': 0.09744062953543126,
 'DSL': 1.0}

In [32]:
# PCA with numeric value
from sklearn.decomposition import PCA

pca1 = PCA(n_components=4)
printcipalComponents1 = pca1.fit_transform(numeric_columns)
principalDf1 = pd.DataFrame(data = printcipalComponents1, columns = ['principal component1', 'principal component2', 'principal component3', 'principal component4'])
# PCA Dataframe

pca1.explained_variance_ratio_

array([0.19661453, 0.18450603, 0.13974742, 0.11021063])

In [33]:
# PCA with categorical value

pca2 = PCA(n_components=1)
printcipalComponents2 = pca2.fit_transform(categorical_columns)
principalDf2 = pd.DataFrame(data = printcipalComponents2, columns = ['principal component1'])
# PCA Dataframe

pca2.explained_variance_ratio_

array([0.47590393])